Last updated by Developer on 2024-06-14.

When populating the similar trade history redis on 2024-02-26, I forgot to remove trades with negative yields before uploading them to the redis. This notebook identifies the trades which may have been added to the similar trades redis, and removes them.

In [18]:
import pandas as pd
import pickle

from main import FEATURES_FOR_EACH_TRADE_IN_HISTORY, \
                 MAX_NUM_TRADES_IN_SIMILAR_TRADE_HISTORY, \
                 similar_trade_history_redis_client, \
                 get_reference_data_from_reference_data_redis, \
                 typecast_reference_data, \
                 left_join_on_cusip, \
                 download_pickle_file, \
                 add_features_for_definition_of_similar, \
                 create_trade_history_numpy_array, \
                 remove_negative_and_missing_yields, \
                 upload_trade_history_to_redis, \
                 upload_to_redis_from_upload_function

Get all the trades for 2024-02-26.

In [2]:
all_trades = download_pickle_file('msrb_intraday_real_time_trade_files', '2024-02-26/all_trade_messages.pkl')
all_trades.head(10)

File 2024-02-26/all_trade_messages.pkl found in msrb_intraday_real_time_trade_files/2024-02-26/all_trade_messages.pkl
Pickle file 2024-02-26/all_trade_messages.pkl downloaded from msrb_intraday_real_time_trade_files/2024-02-26/all_trade_messages.pkl


,message_type,sequence_number,rtrs_control_number,trade_type,transaction_type,cusip,security_description,dated_date,coupon,maturity_date,...,is_weighted_average_price,is_lop_or_takedown,publish_date,publish_time,version,unable_to_verify_dollar_price,is_alternative_trading_system,is_non_transaction_based_compensation,is_trade_with_a_par_amount_over_5MM,upload_date
sequence_number,,,,,,,,,,,,,,,,,,,,,
0,O,0,None,None,None,None,None,None,None,None,...,False,False,None,None,None,False,False,False,False,2024-02-26
1,T,1,2024021600016600,S,R,64971XY28,NEW YORK N Y CITY TRANSITIONAL FIN AUTH REV AD...,2022-08-02,3.05,2052-08-01,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
2,T,2,2024021600049300,S,R,70917RPY5,PENNSYLVANIA ST HIGHER EDL FACS AUTH REV VAR-U...,2008-04-16,3.61,2038-01-01,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
3,T,3,2024021600053100,S,R,64970HAW4,NEW YORK N Y CITY HSG DEV CORP MULTIFAMILY REN...,2002-08-22,3.67,2032-08-15,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
4,T,4,2024021600105400,S,R,64972GZB3,NEW YORK N Y CITY MUN WTR FIN AUTH WTR & SWR S...,2021-03-31,3.1,2045-06-15,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
5,T,5,2024021600108000,P,R,64972GZB3,NEW YORK N Y CITY MUN WTR FIN AUTH WTR & SWR S...,2021-03-31,3.1,2045-06-15,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
6,T,6,2024021600166100,S,R,60528AAS3,MISSISSIPPI BUSINESS FIN CORP MISS GULF OPPORT...,2007-12-19,2.91,2030-12-01,...,False,False,2024-02-26,06:01:01,1.3,False,False,True,False,2024-02-26
7,T,7,2024021600214400,S,R,523470HD7,LEE CNTY FLA ARPT REV AMT-SER B,2021-10-26,4.0,2038-10-01,...,False,False,2024-02-26,06:01:01,1.3,False,False,False,False,2024-02-26
8,T,8,2024021600242900,P,R,13063BFR8,CALIFORNIA ST TAXABLE-VAR PURP,2010-04-01,7.625,2040-03-01,...,False,False,2024-02-26,06:01:01,1.3,False,False,False,False,2024-02-26


Isolate the trades with negative yields.

In [3]:
all_trades_with_negative_yields = all_trades[~pd.isna(all_trades['yield'])]
all_trades_with_negative_yields = all_trades_with_negative_yields[all_trades_with_negative_yields['yield'] < 0]

len(all_trades_with_negative_yields): 59


Add similar trade history redis featuers to get bucket identifiers for trades with negative yields.

In [6]:
cusip_list_from_msrb_data = all_trades_with_negative_yields['cusip'].unique().tolist()    # .unique() prevents same CUSIP being queried in Redis when there are multiple trades with the same CUSIP
reference_data, cusips_not_found = get_reference_data_from_reference_data_redis(cusip_list_from_msrb_data)
reference_data = typecast_reference_data(reference_data)
all_trades_with_negative_yields = left_join_on_cusip(all_trades_with_negative_yields, reference_data)

BEGIN get_reference_data_from_reference_data_redis
END get_reference_data_from_reference_data_redis. Execution time: 0:00:00.140410
BEGIN typecast_reference_data
END typecast_reference_data. Execution time: 0:00:00.000991
BEGIN left_join_on_cusip
END left_join_on_cusip. Execution time: 0:00:00.083512


In [7]:
all_trades_with_negative_yields = all_trades_with_negative_yields.dropna(subset=['issue_key', 'maturity_date', 'trade_date', 'coupon'])
all_trades_with_negative_yields = add_features_for_definition_of_similar(all_trades_with_negative_yields)
print('len(all_trades_with_negative_yields):', len(all_trades_with_negative_yields))

len(all_trades_with_negative_yields): 59


The following function is taken from `fast_trade_history_redis_update/main.py` with minor modifications.

In [8]:
def get_key_trade_history_pair(key, redis_client, max_num_trades, key_transform_func=None, verbose=False, keep_cusip_in_trade_history=False):
    '''`key_transform_func` is helpful in turning a tuple into a primitive type (e.g. string) that can be 
    used as a key for Redis. Redis does not allow tuples to be used as keys.'''
    if key_transform_func is not None: key = key_transform_func(key)
    if verbose: print(f'Calling get_key_trade_history_pair(...) with key={key}')
    features_for_each_trade_in_history = list(FEATURES_FOR_EACH_TRADE_IN_HISTORY.keys())
    if keep_cusip_in_trade_history: features_for_each_trade_in_history.append('cusip')
    if not redis_client.exists(key): print(f'{key} should exist in similar trade history redis')
    old_trade_history = redis_client.get(key)
    try:
        old_trade_history = pd.DataFrame(pickle.loads(old_trade_history), columns=features_for_each_trade_in_history)
    except Exception as e:
        print('key:', key)
        print('old_trade_history:\n', pd.DataFrame(pickle.loads(old_trade_history)))
        raise e
    before_removing_negative_yields = len(old_trade_history)
    old_trade_history = remove_negative_and_missing_yields(old_trade_history)
    after_removing_negative_yields = len(old_trade_history)
    if before_removing_negative_yields == after_removing_negative_yields:
        print('No trades were removed, so no updates need to be made, thus returning `None`')
        return None, None
    return key, create_trade_history_numpy_array(old_trade_history, max_num_trades)

In [10]:
features_to_string = lambda features: '_'.join([str(feature) for feature in features])    # `features` should be a tuple or list; NOTE: this lambda function is identical to `similar_group_to_similar_key(...)` in `app_engine/demo/server/modules/finance.py`
get_features_similar_trade_history_pair_caller = lambda features: get_key_trade_history_pair(features, similar_trade_history_redis_client, MAX_NUM_TRADES_IN_SIMILAR_TRADE_HISTORY, features_to_string, verbose=True, keep_cusip_in_trade_history=True)
features_trade_history_pairs = [get_features_similar_trade_history_pair_caller(features) for features, df_for_features in all_trades_with_negative_yields.groupby(['issue_key', 'years_to_maturity_date_by_5', 'coupon_by_1'])]
features_trade_history_pairs = [(feature, trade_history) for feature, trade_history in features_trade_history_pairs if feature is not None]

Calling get_key_trade_history_pair(...) with key=571357_1_5
Removed 3 trades for having negative or missing yields, leaving 7 trades
Calling get_key_trade_history_pair(...) with key=573112_3_6
Removed 1 trades for having negative or missing yields, leaving 3 trades
Calling get_key_trade_history_pair(...) with key=619626_2_6
Removed 1 trades for having negative or missing yields, leaving 28 trades
Calling get_key_trade_history_pair(...) with key=621809_1_5
Removed 4 trades for having negative or missing yields, leaving 7 trades
Calling get_key_trade_history_pair(...) with key=625701_3_5
Removed 1 trades for having negative or missing yields, leaving 39 trades
Calling get_key_trade_history_pair(...) with key=746450_0_5
Removed 1 trades for having negative or missing yields, leaving 63 trades
Calling get_key_trade_history_pair(...) with key=752411_0_5
Removed 2 trades for having negative or missing yields, leaving 62 trades
Calling get_key_trade_history_pair(...) with key=779986_0_5
Remov

In [13]:
similar_trade_history_redis_client.delete('865275_1_5')    # since this key no longer has any trades, remove it from the redis

1

Upload new trade history pairs to the similar trade history redis.

In [19]:
upload_similar_trade_history_to_similar_trade_history_redis = lambda features, trade_history: upload_trade_history_to_redis(features, trade_history, similar_trade_history_redis_client)
upload_to_redis_from_upload_function(features_trade_history_pairs, upload_similar_trade_history_to_similar_trade_history_redis)

[('571357_1_5',
  array([[Timestamp('2024-02-26 15:19:15'),
          Timestamp('2100-01-01 00:00:00'), 2024022609411100,
          Timestamp('2024-02-26 15:19:04'),
          Timestamp('2024-02-26 15:19:15'), 0.38, 100.381, 10000.0, 'P',
          False, False, None, False, False, datetime.date(2024, 2, 28),
          datetime.date(2024, 3, 28), 0, datetime.date(2033, 2, 15),
          datetime.date(2024, 3, 28), datetime.date(2024, 3, 28), NaT, 'I',
          41729, '795677AG8'],
         [Timestamp('2024-02-13 13:41:15'),
          Timestamp('2100-01-01 00:00:00'), 2024021306627300,
          Timestamp('2024-02-13 13:39:07'),
          Timestamp('2024-02-13 13:41:15'), 5.17, 99.677, 10000.0, 'P',
          False, False, None, False, False, datetime.date(2024, 2, 13),
          datetime.date(2033, 2, 15), 2, datetime.date(2033, 2, 15),
          datetime.date(2024, 3, 15), datetime.date(2024, 3, 15), NaT, 'I',
          29266, '795677AG8'],
         [Timestamp('2024-02-09 08:48:18'),